# Advanced Querying Mongo

Importing libraries and setting up connection

In [2]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [3]:
client.list_database_names()

['Ironhack', 'admin', 'config', 'local']

In [4]:
db = client.get_database("Ironhack")

In [5]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Ironhack')

In [6]:
db.list_collection_names()

['Companies']

In [7]:
c = db.get_collection("Companies")

In [9]:
c.find_one()

{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
 'name': 'Wetpaint',
 'permalink': 'abc2',
 'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
 'homepage_url': 'http://wetpaint-inc.com',
 'blog_url': 'http://digitalquarters.net/',
 'blog_feed_url': 'http://digitalquarters.net/feed/',
 'twitter_username': 'BachelrWetpaint',
 'category_code': 'web',
 'number_of_employees': 47,
 'founded_year': 2005,
 'founded_month': 10,
 'founded_day': 17,
 'deadpooled_year': 1,
 'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
 'alias_list': '',
 'email_address': 'info@wetpaint.com',
 'phone_number': '206.859.6300',
 'description': 'Technology Platform Company',
 'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
 'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
 'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and monetize audiences for di

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [40]:
#list(c.find({"name": "Babelgum"}))

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [56]:
list(c.find({'number_of_employees':{'$gt':5000}}).sort("number_of_employees").limit(2))


[{'_id': ObjectId('52cdef7c4bab8bd675297e49'),
  'name': 'Nintendo',
  'permalink': 'nintendo',
  'crunchbase_url': 'http://www.crunchbase.com/company/nintendo',
  'homepage_url': 'http://nintendo.com',
  'blog_url': '',
  'blog_feed_url': '',
  'twitter_username': 'NintendoAmerica',
  'category_code': 'games_video',
  'number_of_employees': 5080,
  'founded_year': 1889,
  'founded_month': 9,
  'founded_day': 23,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': None,
  'tag_list': 'games, console',
  'alias_list': '',
  'email_address': 'noalegal@noa.nintendo.com',
  'phone_number': '1-800-255-3700 ',
  'description': '',
  'created_at': 'Sat Dec 22 13:27:00 UTC 2007',
  'updated_at': 'Thu Sep 05 12:41:16 UTC 2013',
  'overview': '<p>Nintendo, a technology company widely known for its line of game consoles, was actually founded in 1889 by Fusajiro Yamauchi. It began as a card game company and evolved into one of the largest Japanese c

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [68]:
proy = {'name':1,'founded_year':1,'_id':0}
list(c.find({'founded_year':{'$gte':2000},'founded_year':{'$lte':2005}}, proy).limit(2))

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'AdventNet', 'founded_year': 1996}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [71]:
proy = {'name':1,'ipo':1,'_id':0}
list(c.find({'ipo.valuation_year':{'$gte':100000000},'founded_year':{'$lte':2010}}, proy).limit(2))

[]

In [75]:
cond1 = {"founded_year": {"$lt": 2010}}
cond2 = {"ipo.valuation_amount": {"$gt": 100000000}}
proy = {"name":1, "_id":0, "ipo":1}
x = list(c.find({"$and": [cond1,cond2]},proyec))
x

[{'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'name': 'Yelp',
  'ipo': {'valuation_amount': 1300000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 3,
   'pub_day': 2,
   'stock_symbol': 'NYSE:YELP'}},
 {'name': 'LinkedIn',
  'ipo': {'valuation_amount': 9310000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2011,
   'pub_month': 7,
   'pub_day': 20,
   'stock_symbol': 'NYSE:LNKD'}},
 {'name': 'Amazon',
  'ipo': {'valuation_amount': 100000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 1997,
   'pub_month': 5,
   'pub_day': None,
   'stock_symbol': 'NASDAQ:AMZN'}},
 {'name': 'Brightcove',
  'ipo': {

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [117]:
list(c.find({'number_of_employees':{'$lte':1000},'founded_year':{'$lte':2005}}).sort("number_of_employees").limit(2))

[{'_id': ObjectId('52cdef7c4bab8bd675297dbc'),
  'name': 'Skype',
  'permalink': 'skype',
  'crunchbase_url': 'http://www.crunchbase.com/company/skype',
  'homepage_url': 'http://www.skype.com',
  'blog_url': 'http://blogs.skype.com',
  'blog_feed_url': 'http://feeds.feedburner.com/shareskypeen',
  'twitter_username': 'skype',
  'category_code': 'software',
  'number_of_employees': 0,
  'founded_year': 2003,
  'founded_month': 8,
  'founded_day': None,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': '',
  'tag_list': 'phone, voip, sweden, skype',
  'alias_list': '',
  'email_address': '',
  'phone_number': '94117733',
  'description': 'VoIP Software ',
  'created_at': 'Tue Jul 03 05:20:31 UTC 2007',
  'updated_at': 'Wed Dec 11 13:03:57 UTC 2013',
  'overview': '<p>Skype is a software application that allows users to make voice and video calls and chats over the Internet. Calls to other users within the Skype service are free, while c

### 6. All the companies that don't include the `partners` field.

In [85]:
# $exists te busca si existe el campo, el campo puede estar vacío, pero existe. No hay ningún registro que no tenga ese campo.
cond_exi = {"partners": {"$exists": False}}

list(c.find(cond_exi).limit(2))

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [9]:
#list(c.find({"category_code": {"$type":"null"}}).limit(3))

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [91]:
proy = {'name':1,'number_of_employees':1}
list(c.find({'number_of_employees':{'$gte':100},'number_of_employees':{'$lte':1000}},proy).limit(2))

[{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
  'name': 'Wetpaint',
  'number_of_employees': 47},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8b'),
  'name': 'AdventNet',
  'number_of_employees': 600}]

### 9. Order all the companies by their IPO price in a descending order.

In [115]:
#list(c.find({}).sort("ipo_price",-1).limit(2))

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [116]:
proy= {"name":1,"number of employees":1}
y = list(c.find({},proy).sort("number_of_employees",-1))
#y[:10]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [10]:
proy = {"founded_month": 1, "name": 1}
list(c.find({"founded_month": {"$in": [7,8,9,10,11,12]}}, proy).limit(2))

[{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
  'name': 'Wetpaint',
  'founded_month': 10},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8c'),
  'name': 'Zoho',
  'founded_month': 9}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [98]:
#list(c.find({'acquisition.price_amount':{'$gte':10000},'founded_year':{'$lte':2000}}).limit(2))

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [118]:
proy = {'name':1,'acquisition_field':1}

list(c.find({'acquisition.price_amount':{'$gte':10000},'founded_year':{'$lte':2000}},proy).limit(2))

[{'_id': ObjectId('52cdef7c4bab8bd675297d90'), 'name': 'Postini'},
 {'_id': ObjectId('52cdef7c4bab8bd675297deb'), 'name': 'SideStep'}]

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [121]:
proy = {'name':1,'founded_year':1}
list(c.find({},proy).sort("founded_year",-1).limit(5))

[{'_id': ObjectId('52cdef7c4bab8bd6752982d4'),
  'name': 'Advaliant',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7c4bab8bd675297fec'),
  'name': 'Fixya',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7d4bab8bd675298ea7'),
  'name': 'iBazar',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7c4bab8bd67529830a'),
  'name': 'Fluc',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7c4bab8bd67529801f'),
  'name': 'Wamba',
  'founded_year': 2013}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [125]:
proy = {'name':1,'founded_day':1}
#list(c.find({'founded_day':{'$lte':7}}).sort("acquisition.price_amount",-1).limit(2))


### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [130]:
list(c.find({"category_code":"web"},{"amount_of_employees":{'gte':4000}}).sort("amount_of_employees",-1).limit(2))

[{'_id': ObjectId('52cdef7c4bab8bd675297d8a')},
 {'_id': ObjectId('52cdef7c4bab8bd675297d90')}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [140]:
proy = {"acquisition": 1, "name": 1, }
cond = {"acquisition.price_currency_code": "EUR"}
cond2 = {"acquisition.price_amount": {"$gte": 10000000}}
list(c.find({"$and": [cond,cond2]},proy).sort("number_of_employees",1).limit(2))

[{'_id': ObjectId('52cdef7d4bab8bd675298f47'),
  'name': 'Greenfield Online',
  'acquisition': {'price_amount': 40000000,
   'price_currency_code': 'EUR',
   'term_code': None,
   'source_url': 'http://www.research-live.com/news/m-and-a/toluna-buys-greenfield-online-survey-business-for-$40m/4000346.article',
   'source_description': 'Toluna buys Greenfield Online survey business for $40m',
   'acquired_year': 2009,
   'acquired_month': 6,
   'acquired_day': 26,
   'acquiring_company': {'name': 'Toluna', 'permalink': 'toluna'}}},
 {'_id': ObjectId('52cdef7d4bab8bd675298bf3'),
  'name': 'Apertio',
  'acquisition': {'price_amount': 140000000,
   'price_currency_code': 'EUR',
   'term_code': None,
   'source_url': 'http://www.techcrunch.com/2008/01/03/nokia-siemens-acquires-apertio-for-e140m/',
   'source_description': 'Nokia Siemens Acquires Apertio For â‚¬140m',
   'acquired_year': 2008,
   'acquired_month': 1,
   'acquired_day': 3,
   'acquiring_company': {'name': 'Nokia Siemens Network

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [139]:
proy = {"acquisition": 1, "name": 1}
list(c.find({"acquisition.acquired_month": {"$in": [1,2,3]}}, proy).limit(1))

[{'_id': ObjectId('52cdef7c4bab8bd675297dab'),
  'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}}]

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [11]:
cond1 = {"founded_year": {"$gt": 1999}}
cond2 = {"founded_year": {"$lt": 2011}}
cond3 = {"acquisition.acquired_year": {"$gt": 2011}}
proy = {"name":1, "_id":0, "acquisition.acquired_year":1}
#a = list(c.find({"$and": [cond1,cond2,cond3]},proy))
#a

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code